# 0. 라이브러리 및 데이터 불러오기

* pip install tensorflow

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_digits

import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import random

In [2]:
print("library version check")
print("numpy: ", np.__version__)
print("pandas: ", pd.__version__)
print("seaborn: ", sns.__version__)
print("matplotlib: ", mpl.__version__)
print("sklearn: ", sklearn.__version__)
print("tensorflow: ", tensorflow.__version__)

library version check
numpy:  1.23.5
pandas:  1.5.1
seaborn:  0.12.1
matplotlib:  3.6.2
sklearn:  1.1.3
tensorflow:  2.10.0


In [3]:
matplotlib_inline.backend_inline.set_matplotlib_formats("png2x")
mpl.style.use("default")
mpl.rcParams.update({"figure.constrained_layout.use": True})

sns.set_context("paper") 
sns.set_palette("Set2") 
sns.set_style("whitegrid") 

plt.rc("font", family = "Malgun Gothic")
plt.rcParams["axes.unicode_minus"] = False

# 1. 단층신경망

In [4]:
random.seed(42)

# 입력노드 수 = feature 수
input_node = 4
# 출력노드 수
output_node = 3

#모델
model = Sequential()
model.add(Dense(output_node, 
                activation = "softmax",
                input_shape = (input_node,) 
               ))

$$
\mathbf{w} = \left[
\begin{matrix}
    w_{1,1} & w_{1,2} & w_{1,3} & w_{1,4} \\
    w_{2,1} & w_{2,2} & w_{2,3} & w_{2,4} \\
    w_{3,1} & w_{3,2} & w_{3,3} & w_{3,4} \\
\end{matrix}
\right], \quad
\mathbf{x} = 
\left[
\begin{matrix}
    x_1 \\
    x_2 \\ 
    x_3 \\
    x_4
\end{matrix}
\right],\quad
\mathbf{b} = \left[
\begin{matrix}
    b_1 \\
    b_2 \\ 
    b_3
\end{matrix}
\right]
$$


$$
\mathbf{u} = \mathbf{w}\mathbf{x}+\mathbf{b} = \left[
\begin{matrix}
    w_{1,1} & w_{1,2} & w_{1,3} & w_{1,4} \\
    w_{2,1} & w_{2,2} & w_{2,3} & w_{2,4} \\
    w_{3,1} & w_{3,2} & w_{3,3} & w_{3,4} \\
\end{matrix}
\right]
\left[
\begin{matrix}
    x_1 \\
    x_2 \\ 
    x_3 \\
    x_4
\end{matrix}
\right]
+ \left[
\begin{matrix}
    b_1 \\
    b_2 \\ 
    b_3
\end{matrix}
\right]
$$

$$
\mathbf{z} = f(\mathbf{u})
$$

이 경우는 다중분류이므로 출력층의 활성함수(activattion function)는 softmax이다. 즉, 

$$
f(u_i) = \frac{exp(u_i)}{\sum_{j=1}^Kexp(u_j)}.
$$

여기서, $K$는 클래스의 수이다.

In [5]:
model.get_weights()

[array([[ 0.66323733,  0.30112958, -0.26161683],
        [ 0.8332188 , -0.781279  ,  0.28206468],
        [ 0.8711965 , -0.61490726,  0.0440492 ],
        [ 0.24211967, -0.7658045 , -0.09932661]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [6]:
w = model.get_weights()[0]
b = model.get_weights()[1]
w

array([[ 0.66323733,  0.30112958, -0.26161683],
       [ 0.8332188 , -0.781279  ,  0.28206468],
       [ 0.8711965 , -0.61490726,  0.0440492 ],
       [ 0.24211967, -0.7658045 , -0.09932661]], dtype=float32)

In [7]:
w.T

array([[ 0.66323733,  0.8332188 ,  0.8711965 ,  0.24211967],
       [ 0.30112958, -0.781279  , -0.61490726, -0.7658045 ],
       [-0.26161683,  0.28206468,  0.0440492 , -0.09932661]],
      dtype=float32)

In [8]:
data = np.array([5.1, 3.5, 1.4, 0.2])
data.shape

(4,)

In [9]:
np.reshape(data, (1,4))

array([[5.1, 3.5, 1.4, 0.2]])

In [10]:
model.predict(np.reshape(data, (1,4)))

1/1 [==============================] - 0s 234ms/step


array([[9.995623e-01, 5.656841e-05, 3.810694e-04]], dtype=float32)

In [11]:
np.matmul(data, w)

#같은 결과
#np.matmul(w.T, data)

array([ 7.56687529, -2.21274682, -0.30521588])

$$
\mathbf{u} = \mathbf{w}\mathbf{x}+\mathbf{b} = \left[
\begin{matrix}
    w_{1,1} & w_{1,2} & w_{1,3} & w_{1,4} \\
    w_{2,1} & w_{2,2} & w_{2,3} & w_{2,4} \\
    w_{3,1} & w_{3,2} & w_{3,3} & w_{3,4} \\
\end{matrix}
\right]
\left[
\begin{matrix}
    x_1 \\
    x_2 \\ 
    x_3 \\
    x_4
\end{matrix}
\right]
+ \left[
\begin{matrix}
    b_1 \\
    b_2 \\ 
    b_3
\end{matrix}
\right]
$$

$$
\mathbf{z} = f(\mathbf{u}), 
$$

이 경우는 다중분류이므로 출력층의 활성함수(activattion function)는 softmax이다. 즉, 

$$
f(u_i) = \frac{exp(u_i)}{\sum_{j=1}^Kexp(u_j)}.
$$

여기서, $K$는 클래스의 수이다.

In [12]:
u = np.matmul(w.T, data) + b
u

array([ 7.56687529, -2.21274682, -0.30521588])

In [13]:
def softmax(t):
    return np.exp(t)/np.sum(np.exp(t))

In [15]:
model.predict(np.reshape(data, (1,4)))

1/1 [==============================] - 0s 61ms/step


array([[9.995623e-01, 5.656841e-05, 3.810694e-04]], dtype=float32)

In [14]:
softmax(u)

array([9.99562362e-01, 5.65684108e-05, 3.81069453e-04])

**초기 가중치 문제**
* 가중치 초기화 방법을 따로 설정해 주지 않으면 일정 구간 내에서 랜덤하게 찍는 random_uniform으로 정해짐
* 이 방식은 오차 역전파(back propagation) 과정에서 미분한 gradient가 지나치게 커지거나(exploding gradient) 소실되는(vanishing gradient) 문제가 발생할 수 있음
* 따라서, 어떻게 가중치를 초기화할 것인가에 대한 지속적인 연구가 진행되고 있음

* lecun_uniform, lecun_normal: 98년도에 얀 르쿤(CNN 제안)이 제기한 방법
* glorot_uniform, glorot_normal
* he_uniform, he_normal: ResNet으로도 유명한 마이크로소프트(현재는 Facebook)의 Kaiming He가 2015년에 제안